## Imports

In [1]:
import torch

import networkx as nx

from sklearn.model_selection import train_test_split

In [2]:
from utils.corpus_utils import Corpus, CorpusMaker

## Data opening

In [3]:
filepath = 'data/Bunin-Temnye_allei.txt'

In [4]:
with open(filepath, 'r', encoding='cp1251') as f:
    corpus_raw = f.read().lower()

In [5]:
test_corpus_raw = corpus_raw.split('\n')[57: 56+100]

## Texts to graph

In [6]:
corpus_maker = CorpusMaker()

In [ ]:
%%time
train_graph = corpus_maker.create_graph('lenta')

In [ ]:
test_graph = corpus_maker.create_graph(test_corpus_raw, dicts=(train_graph.idx2token, train_graph.token2idx))

## Saving

In [ ]:
torch.save(train_graph, 'data/train.pickle')
torch.save(test_graph, 'data/test.pickle')

In [ ]:
## Vizualization

In [ ]:
# G = nx.Graph(train_graph['edge_index'].tolist())

In [ ]:
# nx.write_gexf(G, "test.gexf")

In [ ]:
# nx.draw_networkx(G)